In [105]:
import winsound
import pygame
from bs4 import BeautifulSoup, NavigableString
import requests
from time import sleep
from random import randint
import math
from tqdm import tqdm
import pickle
import pandas as pd
from datetime import date
from datetime import datetime
import json

import pymysql                        # for getting data from a SQL database
from sqlalchemy import create_engine, text  # for establishing the connection and authentication
import zlib

from webscraper.scrap_modules import scraper_ds as mybib
from webscraper.scrap_modules import prep as prep

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
# def get_job_information(soup):    
#     class_soup = soup.find("ul", {"class":"description__job-criteria-list"}).find_all('span')
#     list = []
#     for x in class_soup:
#         list.append(x.text.strip())
    
#     return list

In [107]:
# def get_all_job_information(id, response, scrap_date):

#     soup = BeautifulSoup(response.content, "html.parser")
#     job_dict = {}
#     job_dict['id'] = id
#     try:
#         job_dict['title'] = soup.find('h2', class_='top-card-layout__title').text.strip()
#     except:
#         job_dict['title'] = None
#         print("Error: in 'title'")
        
#     try:    
#         job_dict['company'] = soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
#     except:
#         job_dict['company'] = None
#         print("Error: in 'company'")
        
#     try:    
#         job_dict['city'] = soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()
#     except:
#         job_dict['city'] = None
#         print("Error: in 'city'")
#     try:
#         job_dict['posting_date'] = soup.find("span", {"class":"posted-time-ago__text"}).text.strip()
#     except:
#         job_dict['posting_date'] = None
#         print("Error: in 'posting date'")
        
#     try:    
#         job_dict['job_description'] = soup.find("section",{"class":"show-more-less-html"}).text.strip()

#     except:
#         job_dict['job_description'] = None
#         print("Error: in 'job description'")

#     try:    
#         x = get_job_information(soup)        
#         job_dict['seniority_level'] = x[0]
#         job_dict['employment_type'] = x[1]
#         job_dict['job_function'] = x[2]
#         job_dict['industries'] = x[3]
#     except:
#         print(f"Error: get job information in id {id}")
#         job_dict['seniority_level'] = None
#         job_dict['employment_type'] = None
#         job_dict['job_function'] = None
#         job_dict['industries'] = None
    
#     try:    
#         job_dict['scraping_date'] = scrap_date
#     except:
#         job_dict['scraping_date'] = None 
    
#     try:
#         name = job_dict['title']
#         name = name.replace(" ", "%20")
#         company = job_dict['company']
#         company = company.replace(" ", "%20")
#         job_dict['url'] = f"https://www.linkedin.com/jobs/search?keywords={name}%20{company}&location=Berlin%2C%20Berlin%2C%20Germany&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"
#     except:
#         job_dict['url'] = None 
    
                        

#     return job_dict

### load scraped raw data

In [108]:
with open("webscraper/webscrap_data/soup_dict.pkl", "rb") as f:
    scraper_dict = pickle.load(f)

In [109]:
len(scraper_dict)

20061

### transform scraped raw data into df 

In [110]:
df = prep.transform_scrap_data_to_df(scraper_dict)

  2%|▏         | 419/20061 [00:15<09:04, 36.10it/s]  

Error: in 'title'
Error: in 'company'
Error: in 'city'
Error: in 'posting date'
Error: in 'job description'


  2%|▏         | 471/20061 [00:16<09:53, 32.98it/s]

Error: in 'title'
Error: in 'company'
Error: in 'city'
Error: in 'posting date'
Error: in 'job description'


 17%|█▋        | 3354/20061 [01:50<07:57, 34.95it/s]  

...

 99%|█████████▉| 19817/20061 [11:57<00:08, 30.21it/s]

Error: in 'title'
Error: in 'company'
Error: in 'city'
Error: in 'posting date'
Error: in 'job description'
Error: in 'title'
Error: in 'company'
Error: in 'city'
Error: in 'posting date'
Error: in 'job description'
Error: in 'title'
Error: in 'company'
Error: in 'city'
Error: in 'posting date'
Error: in 'job description'


 99%|█████████▉| 19849/20061 [11:58<00:07, 28.54it/s]

Error: in 'title'
Error: in 'company'
Error: in 'city'
Error: in 'posting date'
Error: in 'job description'
Error: in 'title'
Error: in 'company'
Error: in 'city'
Error: in 'posting date'
Error: in 'job description'
Error: in 'title'
Error: in 'company'
Error: in 'city'
Error: in 'posting date'
Error: in 'job description'
Error: in 'title'
Error: in 'company'
Error: in 'city'
Error: in 'posting date'
Error: in 'job description'


100%|██████████| 20061/20061 [12:06<00:00, 27.61it/s]


### second scrap for empty rows

In [111]:
new_df = prep.second_request_empty_values(df)

NameError: name 'requests' is not defined

In [ ]:
# def transform_scrap_data_to_df(dict):
#     columns = ['id','title', 'company', 'city', 'posting_date', 'job_description', 'seniority_level', 'employment_type', 'job_function', 'industries', 'scraping_date', 'url']
#     df = pd.DataFrame(columns=columns)

#     for key in tqdm(dict):
#         id = key
#         response = dict[key]['response']
#         scrap_date = dict[key]["scrap_date"]
#         new_dict = get_all_job_information(id, response, scrap_date)
        
#         new_row_df = pd.DataFrame([new_dict])
#         df = pd.concat([df, new_row_df], ignore_index=True)

#     return(df)

In [ ]:
# scraper_df = transform_scrap_data_to_df(scraper_dict)

In [ ]:
# with open("webscraper/webscrap_data/temp_df.pkl", "wb") as f:
#     pickle.dump(scraper_df, f) 

In [ ]:
# columns = scraper_df.columns

# for col in columns: 
#     print(f"{scraper_df[col].isna().sum()} null values for {col}")
    

### Second request empty rows


In [ ]:
# isnafilter = scraper_df[scraper_df["title"].isna()]
# isnafilter

In [ ]:
# second_request_list = isnafilter["id"].tolist()

In [ ]:
# # second request for empty rows

# second_request_dict = {}

# for id in second_request_list:
#     response = requests.get(f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{id}')
#     wait_time = randint(1,500)
#     sleep(wait_time/1000)

#     dict2 = {}
#     dict2["scrap_date"] = date.today()
# #     dict2["response"] = response
#     second_request_dict[id] = dict2

In [ ]:
# second_request_dict

In [ ]:
# second_request_df = transform_scrap_data_to_df(second_request_dict)
# second_request_df